In [6]:
import sqlite3
import streamlit as st
import pandas as pd

## Creating an SQLite Database

In [7]:
connection = sqlite3.connect("example.db")

In [8]:
connection

In [9]:
cur = connection.cursor()

In [10]:
cur

## Creating a table named 'Movie'

In [11]:
cur.execute("CREATE TABLE Movie(Title TEXT, Year INT, Score REAL)")

OperationalError: table Movie already exists

In [ ]:
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchall()

## Inserting data into 'Movie'

In [ ]:
data = [
    ("The Shawshank Redemption", 1994, 9.3),
    ("The Godfather", 1972, 9.2),
    ("The Dark Knight", 2008, 9.0),
    ("Pulp Fiction", 1994, 8.9),
    ("Schindler's List", 1993, 9.0),
    ("The Lord of the Rings: The Return of the King", 2003, 8.9),
    ("Forrest Gump", 1994, 8.8),
    ("Inception", 2010, 8.8),
    ("Fight Club", 1999, 8.8),
    ("The Matrix", 1999, 8.7),
    ("The Empire Strikes Back", 1980, 8.7),
    ("Interstellar", 2014, 8.6),
    ("Parasite", 2019, 8.5),
    ("Gladiator", 2000, 8.5),
    ("Saving Private Ryan", 1998, 8.6),
    ("The Lion King", 1994, 8.5),
    ("The Green Mile", 1999, 8.6),
    ("Avengers: Endgame", 2019, 8.4),
    ("Braveheart", 1995, 8.4),
    ("The Departed", 2006, 8.5)
]

In [ ]:
cur.executemany("INSERT INTO Movie VALUES(?, ?, ?)", data)

In [ ]:
connection.commit()

In [ ]:
res = cur.execute("SELECT * FROM Movie")
res.fetchall()

In [ ]:
res.description

## Creating dataframe out of the response

In [ ]:
column_names = [desc[0] for desc in res.description]
column_names

In [ ]:
res = cur.execute("SELECT * FROM Movie")
data = res.fetchall()

In [ ]:
data

In [ ]:
df = pd.DataFrame(data=data, columns=column_names)

In [ ]:
df

In [ ]:
connection.close()

In [ ]:
connection = sqlite3.connect("example.db")

In [ ]:
cur = connection.cursor()

In [ ]:
res = cur.execute("SELECT * FROM Movie")
res.fetchall()

In [ ]:
connection.close()

## Retrieve data as a Dataframe given query

In [36]:
import sqlite3
import pandas as pd

def retrieve_data(db_name: str, query: str):
    """
    Function takes in an SQL Query as input and
    returns a pandas dataframe created from the retrieved data
    """
    # Connecting to Database
    connection = sqlite3.connect(db_name)
    # Creating database Cursor
    cur = connection.cursor()
    # Executing SQL query on the databse
    res = cur.execute(query)
    # Comit
    connection.commit()
    # Storing retrieved data
    data = res.fetchall()
    # If query was for retrieval, return dataframe
    if res.description:
        # Retrieving Column names from res if the query was for retrieval
        columns = [desc[0] for desc in res.description]
        # Creating the dataframe
        df = pd.DataFrame(data=data, columns=columns)
        # Closing Connection
        connection.close()
        return df
    # Closing Connection
    connection.close()

In [ ]:
df = retrieve_data("example.db", "SELECT * FROM Movie\n")
df

## Call Gemini API to convert Natural Language to SQL

In [65]:
import os
from dotenv import load_dotenv

import google.generativeai as genai

# Loading all environment variables
load_dotenv()

# Fetching the API KEY
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

system_instruction = """Return only the SQL Query as a string response and nothing else, without any headings, prefixes or suffixes,
                        even if the query is multi-line.
                        Also replace apostrophes within text elements in the query with 2 single quotes, to avoid Syntax Error when
                        executing SQL Commands.
                        You are a data engineer. Your task is to convert natural language queries into SQL Commands,
                        which can be used to query an SQLite database. You will be given table names and columns names.
                        If table name is not given in the prompt, see if the table name was mentioned earlier in the chat history.
                        """
# Setting model to be used
model = genai.GenerativeModel("gemini-1.5-flash",
                             system_instruction=system_instruction)

# starting chat_session
chat_session = model.start_chat()

# Function to convert natural language to SQL
def nl_to_sql(prompt: str, chat=chat_session) -> str:
    sql = chat.send_message(prompt).text
    return sql   

In [2]:
print(nl_to_sql("retrieve all data from table Movie"))

SELECT * FROM Movie



In [3]:
print(nl_to_sql("retrieve first row from that table"))

SELECT * FROM Movie LIMIT 1



In [ ]:
print(nl_to_sql("Sum of values in Score column"))

In [ ]:
nl_to_sql("retrieve all data from table Movie")

## Tying it all together

In [29]:
# Taking user input
user_query = input()

# converting to sql
sql = nl_to_sql(user_query)
# Retriving pandas dataframe
df = retrieve_data("example.db", sql)
df

show all tables


,name
0,Movie
1,House
2,Houses
3,actors
4,One


In [32]:
# Taking user input
user_query = input()

# converting to sql
sql = nl_to_sql(user_query)
print(sql)
# Retriving pandas dataframe
df = retrieve_data("example.db", sql)

df

create new table Two with columns first and second
CREATE TABLE Two (
  first TEXT,
  second TEXT
);



In [33]:
# Taking user input
user_query = input()

# converting to sql
sql = nl_to_sql(user_query)
print(sql)
# Retriving pandas dataframe
df = retrieve_data("example.db", sql)

df

show all tables
SELECT name FROM sqlite_master WHERE type='table'



,name
0,Movie
1,House
2,Houses
3,actors
4,One
5,Two


In [37]:
# Taking user input
user_query = input()

# converting to sql
sql = nl_to_sql(user_query)
print(sql)
# Retriving pandas dataframe
df = retrieve_data("example.db", sql)

df

add values(1,2) to Table Two
INSERT INTO Two (first, second) VALUES ('1', '2');





In [38]:
# Taking user input
user_query = input()

# converting to sql
sql = nl_to_sql(user_query)
print(sql)
# Retriving pandas dataframe
df = retrieve_data("example.db", sql)

df

show all rows in Two
SELECT * FROM Two;





,first,second
0,1,2


In [40]:
# Taking user input
user_query = input()

# converting to sql
sql = nl_to_sql(user_query)
print(sql)
# Retriving pandas dataframe
df = retrieve_data("example.db", sql)

df

add new table named four with columns one and two
CREATE TABLE four (
  one TEXT,
  two TEXT
);





In [53]:
user_query = input()

# converting to sql
sql = nl_to_sql(user_query)
print(sql)
# Retriving pandas dataframe
df = retrieve_data("db1.db", sql)

df

add table named movie1 with columns Title, Year and Score
CREATE TABLE movie1 (
  Title TEXT,
  Year INTEGER,
  Score REAL
);





In [61]:
user_query = input()

# converting to sql
sql = nl_to_sql(user_query)
print(sql)
# Retriving pandas dataframe
df = retrieve_data("db1.db", sql)

df

add following values to the table ("The Shawshank Redemption", 1994, 9.3), ("The Godfather", 1972, 9.2), ("The Dark Knight", 2008, 9.0), ("Pulp Fiction", 1994, 8.9), ("Schindlers List", 1993, 9.0), ("The Lord of the Rings: The Return of the King", 2003, 8.9), ("Forrest Gump", 1994, 8.8), ("Inception", 2010, 8.8), ("Fight Club", 1999, 8.8), ("The Matrix", 1999, 8.7), ("The Empire Strikes Back", 1980, 8.7), ("Interstellar", 2014, 8.6), ("Parasite", 2019, 8.5), ("Gladiator", 2000, 8.5), ("Saving Private Ryan", 1998, 8.6), ("The Lion King", 1994, 8.5), ("The Green Mile", 1999, 8.6), ("Avengers: Endgame", 2019, 8.4), ("Braveheart", 1995, 8.4), ("The Departed", 2006, 8.5)
INSERT INTO movie1 (Title, Year, Score) VALUES
  ('The Shawshank Redemption', 1994, 9.3),
  ('The Godfather', 1972, 9.2),
  ('The Dark Knight', 2008, 9.0),
  ('Pulp Fiction', 1994, 8.9),
  ('Schindlers List', 1993, 9.0),
  ('The Lord of the Rings: The Return of the King', 2003, 8.9),
  ('Forrest Gump', 1994, 8.8),
  ('Incep

In [70]:
user_query = input()

# converting to sql
sql = nl_to_sql(user_query)
print(sql)
# Retriving pandas dataframe
df = retrieve_data("db1.db", sql)

df

detele database db1
DROP DATABASE db1



OperationalError: near "DATABASE": syntax error

In [56]:
connection = sqlite3.connect("db1.db")

In [58]:
cur = connection.cursor()

In [60]:
cur.execute(sql)

OperationalError: near "s": syntax error

In [52]:
user_query = input()

# converting to sql
sql = nl_to_sql(user_query)
print(sql)
# Retriving pandas dataframe
df = retrieve_data("db1.db", sql)

df

show all tables
SELECT name FROM sqlite_master WHERE type='table'



,name
0,movie


In [49]:
# Connecting to Database
connection = sqlite3.connect("example.db")
# Creating database Cursor
cur = connection.cursor()
# Executing SQL query on the databse
res = cur.execute("""CREATE TABLE six (
  
);""")
# Comit
connection.commit()

OperationalError: near ")": syntax error

In [ ]:
from google.generativeai import GenerativeModel, ChatSession

In [12]:
connection = sqlite3.connect("example.db")

In [13]:
connection

In [14]:
cur = connection.cursor()

In [15]:
cur

In [18]:
res = cur.execute("CREATE TABLE Houses(Number TEXT, Year INT, Rooms REAL)")

In [22]:
res.description

In [23]:
connection.close()

In [26]:
res.fetchall()

ProgrammingError: Cannot operate on a closed database.